## Dependencies

Install dependencies not available on Google Collab.
Collab provides numpy, pandas, sklearn, tensorflow, scipy, etc. (see requirements.txt)


In [ ]:
%pip install pinard
%pip install scikeras

## Benchmark details

The results aggregate the combination of the following trainings configurations:

- estimation configuration: [regression, classification]
- datasets configurations: [Single Train, Cross validation with 5 folds and 2 repeats, Augmented Single Train]
- preprocessing configuration: [flat spectrum, savgol, haar, [small set], [big_set]]
- models:
  - for all configuration: BACON, BACON-VG, DECON, PLS(components from 1 to 100), XGBoost, LW-PLS
  - for single train + small_set : Stack > [ BACON, BACON-VG, DECON, PLS(components from 1 to 100), XGBoost, LW-PLS,
    f_PLSRegression,f_AdaBoostRegressor,f_BaggingRegressor,f_ExtraTreesRegressor, f_GradientBoostingRegressor,f_RandomForestRegressor,
    f_ARDRegression,f_BayesianRidge,f_ElasticNet,f_ElasticNetCV,f_HuberRegressor, f_LarsCV,f_LassoCV,f_Lasso,f_LassoLars,f_LassoLarsCV,
    f_LassoLarsIC,f_LinearRegression,f_OrthogonalMatchingPursuit,f_OrthogonalMatchingPursuitCV, f_PassiveAggressiveRegressor,f_RANSACRegressor,
    f_Ridge,f_RidgeCV,f_SGDRegressor,f_TheilSenRegressor,f_GaussianProcessRegressor,f_KNeighborsRegressor, f_Pipeline,f_MLPRegressor,f_LinearSVR,
    f_NuSVR,f_SVR,f_DecisionTreeRegressor,f_ExtraTreeRegressor,f_KernelRidge,f_XGBRegressor]

We perform training in 2 steps, (1) data transformation and (2) training because the sklearn pipeline does not use test data natively.
To change with pinard update in the future.


In [ ]:
### FAST GPU RESET ####
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [4]:
## Browse path and launch benchmark for every folders
%load_ext autoreload
%autoreload 2

from pathlib import Path
from preprocessings import preprocessing_list

from benchmark_loop import benchmark_dataset
import regressors

import tensorflow as tf

tf.get_logger().setLevel("ERROR")
tf.keras.mixed_precision.set_global_policy("mixed_float16")


import logging
logging.basicConfig(
    level= logging.WARNING,
    format="'%(name)s - %(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ]
)


rootdir = Path('data/regression')
folder_list = [f for f in rootdir.glob('**/*') if f.is_dir()]

SEED = ord('D') + 31373

# tf.keras.utils.set_random_seed(SEED)
# tf.config.experimental.enable_op_determinism()


import preprocessings
import regressors
import pinard.preprocessing as pp
from pinard import augmentation, model_selection
from sklearn.cross_decomposition import PLSRegression
from xgboost import XGBRegressor
import sys
import os.path

def str_to_class(classname):
    return getattr(sys.modules['pinard.preprocessing'], classname)

# print(str_to_class('SavitzkyGolay'))


split_configs = [
    None,
    # {'test_size':None, 'method':"random", 'random_state':SEED},
    # {'test_size':None, 'method':"stratified", 'random_state':SEED, 'n_bins':5},
    # {'test_size':0.25, 'method':"spxy", 'random_state':SEED, 'metric':"euclidean", 'pca_components':250},
]

augmentations = [
    None,
    # [(6, augmentation.Rotate_Translate())],
    # [(3, augmentation.Rotate_Translate()),(2, augmentation.Random_X_Operation()),(1, augmentation.Random_Spline_Addition())],
    # [(3, augmentation.Rotate_Translate()),(2, augmentation.Random_X_Operation()),(2, augmentation.Random_Spline_Addition())],
]

preprocessings_list = [
    # None,
    # [("id", pp.IdentityTransformer())],
    # [("baseline", pp.StandardNormalVariate())],
    # [("savgol", pp.SavitzkyGolay())],
    # [("gaussian1", pp.Gaussian(order=1, sigma=2))],
    # [("gaussian2", pp.Gaussian(order=2, sigma=1))],
    # [("haar", pp.Wavelet("haar"))],
    # [("coif3", pp.Wavelet("coif3"))],
    # [("detrend", pp.Detrend())],
    # [("msc", pp.MultiplicativeScatterCorrection(scale=False))],
    # [("dv1", pp.Derivate(1, 1))],
    # [("dv2", pp.Derivate(2, 1))],
    # [("dv3", pp.Derivate(2, 2))],
    
    ("small_set", preprocessings.small_set()),
    ("simple", [("id", pp.IdentityTransformer()), ('haar', pp.Haar()), ('savgol', pp.SavitzkyGolay())]),
    ("transf_set", preprocessings.transf_set()),
    ("bacon_set", preprocessings.bacon_set()),
    ("decon_set", preprocessings.decon_set()),
    # preprocessings.decon_set(),
    # preprocessings.optimal_set_2D(),
    # preprocessings.fat_set(),
]

# for p in preprocessings_list:
#     if isinstance(p, tuple):
#         h = str(hash(frozenset(p[1])))[0:5]
#         print(h)
#     else:
#         h = str(hash(frozenset(p)))[0:5]
#         print(h)


cv_configs = [
    None,
    # {'n_splits':5, 'n_repeats':4},
    # {'n_splits':4, 'n_repeats':2},
    # {'n_splits':3, 'n_repeats':1},
]

# import os
# folder = "data/regression"
# folder = "data/Paprica_2D"
folder = "data/_RefSet"

def get_dataset_list(path, filter=""):
    datasets = []
    for r, d, _ in os.walk(path):
        for folder in d:
            path = os.path.join(r, folder)
            if os.path.isdir(path):
                if str(path).lower()[13] >= filter:
                    datasets.append(str(path))
                # break
    return datasets

folders = get_dataset_list(folder)
# folders = folders[0:1]
# folders = ["data/regression/Cassava_TBC_3556_Davrieux_RMSE1.02"]

len_cv_configs = 0
for c in cv_configs:
    if c == None:
        len_cv_configs += 1
    else:
        len_cv_configs += (c['n_splits'] * c['n_repeats'])

models = [
    # (regressors.ML_Regressor(XGBRegressor), {"n_estimators":200, "max_depth":50, "seed":SEED}),
    # (regressors.ML_Regressor(PLSRegression), {"n_components":50}),

    (regressors.Transformer(), {'batch_size': 10, 'epoch': 1000, 'verbose': 0, 'patience': 100, 'optimizer': 'adam', 'loss': 'mse', "min_lr": 1e-6, "max_lr": 1e-2, "cycle_length": 32}),
    # (regressors.Transformer(), {'batch_size':15, 'epoch':400, 'verbose':0, 'patience':60, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Transformer_VG(), {'batch_size':15, 'epoch':400, 'verbose':0, 'patience':30, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Transformer_NIRS(), {'batch_size':15, 'epoch':400, 'verbose':0, 'patience':30, 'optimizer':'adam', 'loss':'mse'}),
    
    
    # (regressors.FFT_Conv(), {'batch_size':1, 'epoch':10000, 'verbose':0, 'patience':300, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.ResNetV2(), {'batch_size':1, 'epoch':10000, 'verbose':0, 'patience':300, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.XCeption1D(), {'batch_size':1, 'epoch':10000, 'verbose':0, 'patience':600, 'optimizer':'adam', 'loss':'mse'}),
    
    
    # (regressors.Decon_SepPo(), {'batch_size':50, 'epoch':10000, 'verbose':0, 'patience':1000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Decon(), {'batch_size':100, 'epoch':20000, 'verbose':0, 'patience':400, 'optimizer':'adam', 'loss':'mse', "min_lr": 1e-6, "max_lr": 1e-2, "cycle_length": 32}),
    # (regressors.Decon_Sep(), {'batch_size':100, 'epoch':20, 'verbose':2, 'patience':500, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Decon_SepPo(), {'batch_size':100, 'epoch':20000, 'verbose':2, 'patience':500, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Decon_Sep_VG(), {'batch_size':100, 'epoch':20000, 'verbose':2, 'patience':500, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.MLP(), {'batch_size':1000, 'epoch':20000, 'verbose':0, 'patience':2000, 'optimizer':'adam', 'loss':'mse'}),
    # (regressors.Bacon(), {'batch_size':100, 'epoch':10, 'verbose':0, 'patience':10, 'optimizer':'adam', 'loss':'mse'}, True),
]


# from lwpls import LWPLS
# from regressors import NonlinearPLSRegressor

# for i in range(5,150,5):
#     models. append(
#         (regressors.ML_Regressor(NonlinearPLSRegressor, name=f"NL_RBF_PLS_{i}"), {"n_components":i, "poly_degree":2, "gamma":0.1})
#     )
#     models. append(
#         (regressors.ML_Regressor(PLSRegression, name=f"PLS_{i}"), {"n_components":i})
#     )

# models.append(
#     (regressors.ML_Regressor(LWPLS, name=f"LWPLS_0-05_45"), {"max_component_number":45, "lambda_in_similarity":0.05})
# )

# for i in range(10,100,50):
#     models.append(
#         (regressors.ML_Regressor(LWPLS, name=f"LWPLS_0-1_{i}"), {"max_component_number":i, "lambda_in_similarity":0.05})
#     )
    # models.append(
    #     (regressors.ML_Regressor(LWPLS, name=f"LWPLS_0-5_{i}"), {"max_component_number":i, "lambda_in_similarity":0.5})
    # )
# models.append( (regressors.ML_Regressor(PLSRegression, name=f"PLS_5"), {"n_components":5}) )


# for i in range(1,20,1):
#     models.append( (regressors.ML_Regressor(PLSRegression, name=f"PLS_{i}"), {"n_components":i}) )
# for i in range(21,120,3):
#     models.append( (regressors.ML_Regressor(PLSRegression, name=f"PLS_{i}"), {"n_components":i}) )
# for i in range(1,50,10):
#     models.append( (regressors.ML_Regressor(XGBRegressor, name=f"XGB_{i}"), {"n_estimators":i, "max_depth":max(30, int(i/2)), "seed":SEED}) )

# models.append((regressors.Decon(), {'batch_size':100, 'epoch':20000, 'verbose':0, 'patience':400, 'optimizer':'adam', 'loss':'mse'}))
# benchmark_size = len(folders) * len(split_configs) * len_cv_configs * len(augmentations) * len(preprocessings_list) * len(models)
# print("Benchmarking", benchmark_size, "runs.")
# benchmark_dataset(folders[:-1], split_configs, cv_configs, augmentations, preprocessings_list, models, SEED) #, resampling='resample', resample_size=2048) #bins=5)


# benchmark_dataset([folders[12]], split_configs, cv_configs, augmentations, preprocessings_list, models, SEED)
for folder in folders[:-1]:
    print("adding folder", folder)
    benchmark_dataset([folder], split_configs, cv_configs, augmentations, preprocessings_list, models, SEED)


# for folder in folders[:-1]:
#     print("adding folder", folder)
#     benchmark_dataset([folders[i]], split_configs, cv_configs, augmentations, preprocessings_list, [(regressors.CONV_LSTM(), {'batch_size':10, 'epoch':1, 'verbose':0, 'patience':2000, 'optimizer':'adam', 'loss':'mse'}),], SEED)
    

# import threading
# import queue

# def worker(q):
#     while True:
#         item = q.get()
#         if item is None:
#             break
#         # process item
#         # print(f'Processing {item}')
#         benchmark_dataset(*item)
#         q.task_done()

# q = queue.Queue()
# num_worker_threads = 1
# threads = []
# for i in range(num_worker_threads):
#     t = threading.Thread(target=worker, args=(q,))
#     t.start()
#     threads.append(t)

# # add items to the queue
# for folder in folders[:-1]:
#     print("adding folder", folder)
#     q.put(([folder], split_configs, cv_configs, augmentations, preprocessings_list, models, SEED))

# # block until all tasks are done
# q.join()

# # stop workers
# for i in range(num_worker_threads):
#     q.put(None)


# for t in threads:
#     t.join()


print("Benchmarking finished.")



# benchmark_dataset(folders, split_configs, cv_configs, augmentations, preprocessings_list, models, SEED) #, resampling='resample', resample_size=2048) #bins=5)


# benchmark_dataset(folders, split_configs, cv_configs, augmentations, preprocessings_list, models, SEED, resampling='crop', resample_size=2150)


# for folder in folder_list:
    # # print(ord(str(folder)[17]), ord('A'), ord('M'))
    # if ord(str(folder)[16]) < ord("L") or ord(str(folder)[16]) > ord("M"):
    #     continue
    # benchmark_dataset(folder, SEED, preprocessing_list(), 20, augment=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
adding folder data/_RefSet\ALPINE_Calpine_424_Murguzur_RMSE1.36
========== ALPINE_CALPINE_424_MURGUZUR_RMSE1.36 

'root - 2023-06-08 19:06:37,381 [WARNING] Delimiter correction for data\_RefSet\ALPINE_Calpine_424_Murguzur_RMSE1.36\Ycal.csv.gz: \n
'root - 2023-06-08 19:06:37,385 [WARNING] No y file y_test - *Ytest* found for ALPINE_Calpine_424_Murguzur_RMSE1.36.
'root - 2023-06-08 19:06:37,386 [WARNING] Delimiter correction for data\_RefSet\ALPINE_Calpine_424_Murguzur_RMSE1.36\Yval.csv.gz: \n


========== (272, 2151) (272, 1) (152, 2151) (152, 1)
Finished ALPINE_Calpine_424_Murguzur_RMSE1.36 with 5 results
adding folder data/_RefSet\ALPINE_C_424_Murguzur_RMSE1.16
========== ALPINE_C_424_MURGUZUR_RMSE1.16 

'root - 2023-06-08 19:06:38,451 [WARNING] Delimiter correction for data\_RefSet\ALPINE_C_424_Murguzur_RMSE1.16\Ycal.csv.gz: \n
'root - 2023-06-08 19:06:38,455 [WARNING] No y file y_test - *Ytest* found for ALPINE_C_424_Murguzur_RMSE1.16.
'root - 2023-06-08 19:06:38,457 [WARNING] Delimiter correction for data\_RefSet\ALPINE_C_424_Murguzur_RMSE1.16\Yval.csv.gz: \n


========== (361, 2151) (361, 1) (63, 2151) (63, 1)
Finished ALPINE_C_424_Murguzur_RMSE1.16 with 5 results
adding folder data/_RefSet\ALPINE_Nalpine_552_Murguzur_RMSE0.38
========== ALPINE_NALPINE_552_MURGUZUR_RMSE0.38 

...

'root - 2023-06-08 19:06:54,656 [WARNING] Delimiter correction for data\_RefSet\Cassava_TTC_3830_Davrieux_RMSE1.38\Ycal.csv.gz: \n
'root - 2023-06-08 19:06:54,664 [WARNING] No y file y_test - *Ytest* found for Cassava_TTC_3830_Davrieux_RMSE1.38.
'root - 2023-06-08 19:06:54,666 [WARNING] Delimiter correction for data\_RefSet\Cassava_TTC_3830_Davrieux_RMSE1.38\Yval.csv.gz: \n


========== (3368, 1050) (3368, 1) (462, 1050) (462, 1)
RUN Cassava_TTC_3830_Davrieux_RMSE1.38 Transformer transf_set
--- Trainable: 9405 - untrainable: 0.0 > 9405.0


'absl - 2023-06-08 19:06:56,294 [WARNING] Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, query_layer_call_fn, query_layer_call_and_return_conditional_losses, key_layer_call_fn while saving (showing 5 of 14). These functions will not be directly callable after loading.


********** (462, 1050) (462, 1) (462, 1)        
Epoch: 0 > RMSE: 5.1518655  ( 1.5740196 | Transformer--NoSpl-NoCV-Fold_1(1)-NoAug-small_set-31441-23-06-08_17-55-33 ) - R²: -2.356924436000692  val_loss inf
Best so far > 5.1518655 results\Cassava_TTC_3830_Davrieux_RMSE1.38\Transformer--NoSpl-NoCV-Fold_1(1)-NoAug-transf_set-31441-23-06-08_19-06-54
********** (462, 1050) (462, 1) (462, 1)0 0.041572           
Epoch: 1 > RMSE: 2.4289222  ( 1.5740196 | Transformer--NoSpl-NoCV-Fold_1(1)-NoAug-small_set-31441-23-06-08_17-55-33 ) - R²: 0.2538265905139906  val_loss 0.04157228022813797
Best so far > 2.4289222 results\Cassava_TTC_3830_Davrieux_RMSE1.38\Transformer--NoSpl-NoCV-Fold_1(1)-NoAug-transf_set-31441-23-06-08_19-06-54
********** (462, 1050) (462, 1) (462, 1) 0.009260           
Epoch: 2 > RMSE: 2.0126436  ( 1.5740196 | Transformer--NoSpl-NoCV-Fold_1(1)-NoAug-small_set-31441-23-06-08_17-55-33 ) - R²: 0.4876739931183056  val_loss 0.009260456077754498
Best so far > 2.0126436 results\Cassava_

KeyboardInterrupt: 

In [ ]:
# browse all folders in "results" folder
# load the _runs.txt and print the last line
import os
for r, d, _ in os.walk("results"):
    for folder in d:
        path = os.path.join(r, folder)
        if os.path.isdir(path):
            with open(os.path.join(path, "_runs.txt"), "r") as f:
                lines = f.readlines()
                print(folder, lines[-1].strip())
                print([round(float(l.split("  ")[0]), 3) for l in lines])

In [ ]:
# import minmaxscaler
from sklearn.preprocessing import MinMaxScaler
import sklearn

minMaxScaler = MinMaxScaler(feature_range=(0, 0.5))
print(type(minMaxScaler), minMaxScaler)

In [ ]:
from itertools import product


def generate_combinations(steps):
    methods = []
    for step in steps.values():
        methods.append(step['methods'])
    return list(product(*methods))


pre_indexation_steps = {
    "step_1": {
        "type": "filter",
        "methods": ['method_1', 'method_2'],  # List[List[(TransformerMixin, Dict)]]
    },
    "step_2": {
        "type": "augmentation",
        "methods": ['method_3', 'method_4'],  # List[List[(TransformerMixin, Dict)]]
    },
}

combinations = generate_combinations(pre_indexation_steps)
print(combinations)

In [ ]:
import joblib
from data import load_data
from benchmark_loop import get_datasheet
from scikeras.wrappers import KerasRegressor
from sklearn.pipeline import Pipeline
import glob
import json

dataset = "Paprica_XY2_eqGlu"
canon_folder = "results/" + dataset + "/"
json_file = canon_folder[:-1] + "_results.json"
# sort json_file dict by "RMSE" key
with open(json_file, 'r') as f:
    json_dict = json.load(f)
    json_dict = {k: v for k, v in json_dict.items() if "type" in v}
    json_dict = dict(
        sorted(json_dict.items(), key=lambda item: float(item[1]["RMSE"])))

    if len(json_dict) == 0:
        print("no best model saved")
    else:
        run = list(json_dict.values())[0]
        print(json.dumps(run, indent=4))

        X, y, X_valid, y_valid = load_data(run["path"])
        print(X.shape, y.shape, X_valid.shape, y_valid.shape)

        canon_name = canon_folder + run["model"]
        print(canon_name)

        transformer_pipeline = joblib.load(canon_name + "_tf.pkl")
        y_scaler = joblib.load(canon_name + "y_scaler.pkl")
        model = None

        if run["type"] == "NN":
            regex = os.path.join(canon_name + '*' + '.h5')
            weight_files = glob.glob(regex)
            weight_file = max(weight_files, key=os.path.getctime)
            model = tf.keras.models.load_model(weight_file)
        else:
            model = joblib.load(canon_name + "_reg.pkl")

        X_valid = transformer_pipeline.transform(X_valid)
        y_pred = model.predict(X_valid)
        y_pred = y_scaler.inverse_transform(y_pred)

        datasheet = get_datasheet(
            "dataset_name", "model_name", "path", SEED, y_valid, y_pred)
        print(json.dumps(datasheet, indent=4))

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import pandas as pd
import os
import re
from scipy import signal
path = "Data/Paprica_2D/Paprica_2D_XY_ag"
projdir = Path(path)
# print(projdir.glob("*XCal*"))
# for t in projdir.glob("*XCal*"):
    # print(t)
files = tuple(next(projdir.glob(n)) for n in ["*Xcal*", "*Ycal*"])
print(files)

# from data import load_multiple_data
# load_multiple_data()

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
y_train = np.array([0, 1, 2, 2.5, 3, 3.5, 6, 8, 20]).reshape(-1, 1)
bins = 4
discretizer = KBinsDiscretizer(
    n_bins=bins, encode='onehot-dense', strategy='uniform')
discretizer.fit(y_train)
tt = discretizer.transform(y_train)
print(tt)

In [ ]:
import json
from numpy import genfromtxt
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict
import numpy as np


def plot_regression_results(ax, y_true, y_pred, title, scores, elapsed_time):
    """Scatter plot of the predicted vs true targets."""
    ax.plot(
        [y_true.min(), y_true.max()], [y_true.min(), y_true.max()], "--r", linewidth=2
    )
    ax.scatter(y_true, y_pred, alpha=0.2)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines["left"].set_position(("outward", 10))
    ax.spines["bottom"].set_position(("outward", 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel("Measured")
    ax.set_ylabel("Predicted")
    extra = plt.Rectangle(
        (0, 0), 0, 0, fc="w", fill=False, edgecolor="none", linewidth=0
    )
    ax.legend([extra], [scores], loc="upper left")
    title = title + "\n Evaluation in {:.2f} seconds".format(elapsed_time)
    ax.set_title(title)


def plot_data(d, filepath):
    plt.scatter(d[:, 0], d[:, 1])
    plt.xlabel('test')
    plt.ylabel('predict')
    plt.savefig(filepath + '.png')
    plt.close()


path = 'results'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            filepath = os.path.join(root, file)
            df = pd.read_csv(filepath)
            my_data = genfromtxt(filepath, delimiter=';')
            # print(my_data)
            plot_data(my_data, filepath.replace('csv', 'png'))
        # if file.endswith('.json'):
        #     print(file)
        #     dataset = file.replace('.json','')
        #     f = open(os.path.join(root, file))
        #     data = json.load(f)
        #     for key in data:
        #         print(key)

# returns JSON object as 
# a dictionary
            # filepath = os.path.join(root, file)
            # df = pd.read_csv(filepath)
            # y_res = df.iloc[:,0]
            # y_pred = df.iloc[:,1]
            # fig, axs = plt.subplots(1,1, figsize=(10,10))
            # axs = np.ravel(axs)

            # plot_regression_results(
            #     ax,
            #     y,
            #     y_pred,
            #     name,
            #     (r"$R^2={:.2f} \pm {:.2f}$" + "\n" + r"$MAE={:.2f} \pm {:.2f}$").format(
            #         np.mean(score["test_r2"]),
            #         np.std(score["test_r2"]),
            #         -np.mean(score["test_neg_mean_absolute_error"]),
            #         np.std(score["test_neg_mean_absolute_error"]),
            #     ),
            #     elapsed_time,
            # )


# plt.suptitle("Single predictors versus stacked predictors")
# plt.tight_layout()
# plt.subplots_adjust(top=0.9)
# plt.show()

In [ ]:
from pinard.utils import load_csv
from benchmark_loop import transform_test_data
import preprocessings
import numpy as np

dumb_set = preprocessings.dumb_and_dumber_set()
Xfile = "data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36/Xcal.csv.gz"
yfile = "data/regression/ALPINE_Calpine_424_Murguzur_RMSE1.36/Ycal.csv.gz"
X_train, y_train = load_csv(Xfile, yfile, x_hdr=0, y_hdr=0, sep=";")
X_train, y_train, X_test, y_test = X_train[0:
                                           100], y_train[0:100], X_train[0:100], y_train[0:100]
X_test_pp, y_test_pp, transformer_pipeline, y_scaler = transform_test_data(
    dumb_set, X_train, y_train, X_test, y_test, type="augmentation")

print(X_test_pp.shape)
sample = X_test_pp[0]
print(len(dumb_set))
ok = []
for i in range(len(dumb_set)-1, -1, -1):
    found = False
    for j in range(i-1, -1, -1):
        if np.allclose(sample[i], sample[j], rtol=10e-3, atol=10e-3):
            found = True
            break
    if not found:
        ok.append(dumb_set[i][0])

print(len(ok))
print(ok)

In [ ]:
import os
import csv


def count_columns(file):
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=";")
        header = next(reader)
        header = next(reader)
        return len(header), header


def walk_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                path = os.path.join(root, file)
                columns, header = count_columns(path)
                if columns > 2100 and columns < 4000:
                    print(header[0:5])
                    print(f"File: {path}, Columns: {columns}")


# Replace '.' with the directory path you want to traverse
walk_directory('rawdata/regression')

In [ ]:
import csv
import json
import os


def json_to_csv(json_folder, csv_file):
    csv_rows = []
    for file in os.listdir(json_folder):
        if file.endswith(".json"):
            file_path = os.path.join(json_folder, file)
            with open(file_path, "r") as f:
                json_data = json.load(f)
            for key, obj in json_data.items():
                csv_row = {}
                for key, value in obj.items():
                    csv_row[key] = value
                csv_rows.append(csv_row)

    fieldnames = []
    for row in csv_rows:
        for key in row.keys():
            if key not in fieldnames:
                fieldnames.append(key)

    with open(csv_file, "w") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in csv_rows:
            writer.writerow(row)


json_folder = "results"
csv_file = "result.csv"

json_to_csv(json_folder, csv_file)

In [ ]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check if TensorFlow can access the GPU
print("Is GPU available:", tf.config.list_physical_devices('GPU'))

# Run a simple computation on the GPU (if available) and time it
with tf.device('/gpu:0'):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    c = tf.matmul(a, b)

# Print the result
print(c)